In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import re

llm = ChatOpenAI(
    api_key="sk-EQY6oy2D9Brb2IWxymHTnmeFz5unIyobzHtjDgFs2ZwEPjmY",
    base_url="https://api.chatanywhere.tech/v1",
    model="gpt-4o-mini",
    temperature=0.35,
)
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)
def get_files(path):
    import os
    files = os.listdir(path)
    return [os.path.join(path, file) for file in files if file.endswith('.txt')]

In [2]:
def clean_text(text):
    # 替换多个空格为单个空格
    text = re.sub(r" +", " ", text)
    # 替换多个换行+空格为单个换行
    text = re.sub(r"\n\s+", "\n", text)
    return text.strip()  # 去除首尾空格或换行

def split_by_title(text):
    pattern = r"(?:(?<=\n)|(?<=\s)|(?<=^))([一二三四五六七八九十])、"
    parts = re.split(pattern, text)
    chunks = []
    for i in range(1, len(parts), 2):
        tmp = parts[i] + '、' + parts[i + 1]
        tmp = clean_text(tmp).replace(" \n", "\n")
        chunks.append(tmp)    
    return chunks

def get_chapter_title(text):
    pattern = r"(第[一二三四五六七八九十]+节)[、，：:]?\s*(.+)"
    match = re.search(pattern, text)
    return match.group(1) + " " + match.group(2).strip() if match else None

def split_by_pattern(text, pattern):
    parts = re.split(pattern, text)
    chunks = []
    for i in range(1, len(parts), 2):
        tmp = parts[i] + parts[i + 1]
        tmp = clean_text(tmp).replace(" \n", "\n").replace('（', '(').replace('）', ')')
        chunks.append(tmp)
    return chunks

In [3]:
chapter_files = get_files(r'D:\code\agent_demo\kg\clear_out_v3')
chapter_chunks = []
for chapter_file in chapter_files:
    with open(chapter_file, 'r', encoding='utf-8') as f:
        chapter_text = f.read()
    chapter_title = get_chapter_title(chapter_text).replace('\n', '')
    chapter_text = chapter_text.replace(chapter_title, '')
    chunks = split_by_title(chapter_text)
    # print(chunks)
    chapter_chunks.append((chapter_title, chunks))

In [4]:
# 保存
# with open('chapter_chunks.txt', 'w', encoding='utf-8') as f:
#     for chapter_title, chunks in chapter_chunks:
#         f.write(chapter_title + '\n')
#         for chunk in chunks:
#             f.write(chunk + '\n')
#         f.write('\n')
chapter_chunks

[('第一节 教育制度概述',
  ['一、制度的概念 【单选】\n关于制度的内涵中外学者各抒己见，有多种说法。综合中西方关于制度的认识，我们认为制度指为了满足人们的生存与发展的需要、在基于权利和义务的基础上形成的社会关系，以及与此相关的社会活动的规范系统。\n制度实际上包含三层含义：\n①指社会形态意义上的制度，这是对制度的宏观分析。如奴隶制度、封建制度、资本主义制度等，以整个社会作为实体，在区分社会发展类型、性质和阶段时使用。\n②指社会中一些具体制度，这是对社会制度的中观分析。如一个社会内部的家庭、经济、政治、教育、宗教制度等，它以具体的社会组织机构、制度设施作为实体，常在分析不同的社会关系、研究不同社会生活领域的具体问题时使用。\n③指社会规范、规则意义上的制度，是对制度的微观分析。如教育制度内部的考试制度、学习制度等，它以具体社会单位的行为规范、行为模式为实体，常在讨论个体行为时使用。',
   '二、教育制度的定义与构成要素 【单选、填空】\n(一)教育制度的定义\n教育制度是一个应用十分广泛的概念，研究者给出的定义也丰富多彩。\n(1)《教育大辞典》的界定是“一个国家各种教育机构的体系，包括学校制度和管理学校的教育行政机构体系”。\n(2)《中国大百科全书》教育卷对教育制度作出了两种阐释：一是“根据国家的性质制定的教育目的、方针和设施的总称”；二是“各种教育机构系统”。\n(3)在大陆和台湾学者联合编写的《教育百科辞典》中，教育制度有两种解释：①根据国家的性质制定的教育目的、方针和设施的总称。②一个国家内各种教育机构的体系：一是泛指有组织的教育和教学的机构体系，包括学前教育机构、各级各类学校教育机构、成人教育机构、少年儿童校外教育机构以及各级教育行政组织机构；二是专指各级各类学校教育制度，简称学制。\n(二)教育制度的构成要素\n制度是由组成制度的各个要素相互影响、作用与结合而构成的有机整体。关于制度的结构，许多学者给出了不同的答案。有研究者认为制度包括制度目标、制度规则、制度组织和制度载体四个基本要素。而诺斯根据制度的不同表现形式，将制度分为正式约束的规则、非正式约束的规则和实施机制三个部分。我们认为，作为一种规范系统，教育制度的构成要素主要包括：规则要素、物质要素和组织要素。\n1.规则要素\n规则要素主要指教育制度是一个规范系统，用以规范学校成员

In [5]:
# 识别文本中各级小标题
# 0 一、 xxx + '\n' 或 ' '
# 1 (一) xxx + '\n' 或 ' '
# 2 1. xxx + '\n' 或 ' '
# 3 (1) xxx + '\n' 或 ' '
# 识别后，将文本分割为各级小标题和内容 转化为一下列表
# 类别（标题或内容） 标题文本/内容文本 + 标题级别（如果有）

In [4]:
# import re


# def parse_text_structure(text):
#     # 正则模式匹配各级标题，匹配到换行前
#     patterns = [
#         (r"^(一|二|三|四|五|六|七|八|九|十|百|千)[、\s](.*)", 0),  # 一级标题
#         (r"^\(\s*(一|二|三|四|五|六|七|八|九|十)\s*\)\s*(.*)", 1),  # 二级标题
#         (r"^(\d+)\.\s*(.*)", 2),  # 三级标题
#         (r"^\(\s*(\d+)\s*\)\s*(.*)", 3)  # 四级标题
#     ]

#     result = []
#     lines = text.strip().split("\n")
#     current_content = []

#     for line in lines:
#         line = line.strip()
#         if not line:
#             continue

#         matched = False
#         for pattern, level in patterns:
#             match = re.match(pattern, line)
#             if match:
#                 # 处理前面累积的内容
#                 if current_content:
#                     result.append(("内容", "\n".join(current_content), -1))
#                     current_content = []
                
#                 title_text = match.group(2).strip()
#                 result.append(("标题", title_text, level))
#                 matched = True
#                 break
        
#         if not matched:
#             current_content.append(line)

#     # 处理剩余的内容
#     if current_content:
#         result.append(("内容", "\n".join(current_content), -1))

#     return result


# # 示例文本
# text = chapter_chunks[2][1][0]

# # 解析
# parsed_data = parse_text_structure(text)

# # 打印结果
# for item in parsed_data:
#     print(item)


('标题', '教育政策概述 【单选、填空、简答】', 0)
('标题', '教育政策的概念', 1)
('内容', '所谓“政策”，是指国家和政党为实现一定历史时期的任务和目标而规定的行动依据和准则。由此出发，我们可以做出如下定义:教育政策是一个有目的、有组织的动态发展过程，是政党、政府等政治实体在一定历史时期，为了实现一定的教育目标和任务而平衡各方教育利益、协调教育关系所规定的行动依据和准则。\n教育政策具体包含四个方面的内容。', -1)
('标题', '教育政策的决策主体', 2)
('内容', '这是教育政策的所属问题，即“是谁的政策，是谁制定的政策”问题。新中国成立之初，我国普遍习惯于将教育政策看作党的政策，教育政策的主体是中国共产党，这是由当时政府教育管理经验不足决定的。随着政治体制改革的深入推进，教育政策的主体开始由单一主体(中央和地方党委)向复合主体(中央和地方党委、人民代表大会、政府)转变。因此，我国教育政策的决策主体包括：中央和地方各级党委、各级人民代表大会、政府。', -1)
('标题', '教育政策的客体', 2)
('内容', '即教育政策的作用对象。主要解决教育政策的目标和适用范围，回答对谁或对什么事物产生影响的问题。政策客体既可以是人，如《关于严禁使用童工的通知》、残疾人教育条例等所规范的对象；也可以是事，如《中华人民共和国教育法》《中华人民共和国民办教育促进法》规范的对象是教育事业是否能够健康有序发展问题；还可以是物，如《教育部部属高等院校物资仓库管理办法》中小学校财务管理制度等规范的是物资设备。', -1)
('标题', '教育政策的表现形态', 2)
('内容', '它既可以是静态的结果，也可以是教育政策的运行过程。通常人们习惯于将教育政策理解为静态的，如纲要、规划、条例、决定、通知、规定、意见、办法、细则等，而容易忽视教育政策的动态性，即其运行情况。事实上，教育政策的动态运行过程是构成教育政策的重要部分，忽视这一点，将不利于教育政策的理解、贯彻和客观评价。', -1)
('标题', '教育政策的作用方式', 2)
('内容', '不同的观点认为：教育政策是国家教育活动的方向和教育发展的目标；教育政策是教育事业的目标；教育政策是对教育事业的统筹规划等。比较有代表性的观点认为，教育政策是一种行动依据和准则。教育政策作为一种行动依据

In [5]:
title_counter = 1  # 用来给标题编号
def parse_text_structure(text):
    # 定义匹配不同级别标题的正则表达式
    patterns = [
        (r"^(一|二|三|四|五|六|七|八|九|十|百|千)[、\s](.*)", 0),  # 一级标题
        (r"^\(\s*(一|二|三|四|五|六|七|八|九|十)\s*\)\s*([^ 。\n：]*)", 1),  # 二级标题，匹配到空格、句号或换行
        (r"^(\d+)\.\s*([^ 。\n：]*)", 2),  # 三级标题
        (r"^\(\s*(\d+)\s*\)\s*([^ 。\n：]*)", 3)  # 四级标题
    ]

    result = []
    lines = text.strip().split("\n")
    current_title = None

    global title_counter  # 使用全局变量
    level_stack = []  # 存储各级标题的编号

    current_title = None

    for line in lines:
        line = line.strip()
        if not line:
            continue  # 跳过空行

        matched = False
        for pattern, level in patterns:
            match = re.match(pattern, line)
            if match:
                title_text = match.group(2).strip()  # 仅提取标题内容
                
                # 找到最近的上级标题
                while level_stack and level_stack[-1][1] >= level:
                    level_stack.pop()
                previous_title = level_stack[-1][0] if level_stack else None

                # 添加标题信息
                result.append(("标题", title_counter, title_text, level, previous_title))
                current_title = title_counter  # 设置当前标题编号
                title_counter += 1  # 增加编号
                
                # 记录当前标题到层级栈
                level_stack.append((current_title, level))

                # 为当前标题添加空内容
                result.append(("内容", re.sub(pattern, "", line), current_title))
                matched = True
                break
        
        if not matched:
            if result and result[-1][0] == "内容" and result[-1][2] == current_title:
                # 追加到上一条内容
                result[-1] = ("内容", result[-1][1] + "\n" + line, current_title)
            else:
                # 新的内容块
                result.append(("内容", line, current_title))

    return result

In [6]:
import os
from langchain_neo4j import Neo4jGraph

os.environ["NEO4J_URI"] = "neo4j+s://5f494a0d.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "JPvH9AyoJVJDlbOzUml3qfyA_oWutzcMfQCJI9qRCOY"

graph = Neo4jGraph()

d:\Python\MiniConda\envs\itext2kg\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def get_decs_from_text(title, text):
    # 从文本中提取描述
    decs_sys_template = """
-目标- 
给定一个标题和与标题相关的文本文档，以不超过50字描述文档内容，为标题提供一个简短的描述。

-注意事项-
确保描述简洁明了、不存在指代混乱以及冗余。

-示例-
######################
示例1：
标题：自然成熟理论
文本：自然成熟理论的代表人物是格赛尔。他认为支配儿童发展的因素是成熟和学习，他更着重于成熟对儿童发展的制约，认为成熟与主体的内环境有关，而学习则与客观外环境有关。儿童心理发展是儿童行为或心理形式在环境的影响下按一定顺序出现的过程，这个顺序与成熟关系较大，而与外环境关系较小，外环境只是给发展提供适应的时机而已。格赛尔指出无视成熟的学习和训练是徒劳的，是没有成效的，特别是人的早期发展，对内部成熟的依赖程度大大高于外部环境的学习和影响。他的“双生子爬梯试验”提供了儿童发展中成熟优势的实证，为大家所公认。此实验说明了成熟是某些行为、能力产生的必需条件，是身心发展的一种准备状态。在某些人体机能的生理结构未成熟之前，学习和训练的成效是很差的，甚至是不可能进行的，只有在某一行为出现的机体发育临近或达到成熟状态时，学习和训练才能奏效。
——————————————————————
输出：自然成熟理论由格赛尔提出，强调儿童发展主要受生理成熟驱动，外部环境仅提供适应机会。
######################
"""
    decs_sys_prompt = SystemMessagePromptTemplate.from_template(decs_sys_template)

    decs_human_template="""
    -真实数据- 
    ######################
    标题：{title}
    文本：{input_text} 
    ###################### 
    输出：
    """

    from langchain.chains import LLMChain
    chat_prompt = ChatPromptTemplate.from_messages([decs_sys_prompt, decs_human_template])
    llm = ChatOpenAI(
        api_key="sk-EQY6oy2D9Brb2IWxymHTnmeFz5unIyobzHtjDgFs2ZwEPjmY",
        base_url="https://api.chatanywhere.tech/v1",
        model="gpt-4o-mini",
        temperature=0.35,
    )
    chain = LLMChain(llm=llm, prompt=chat_prompt)
    
    answer = chain.invoke({
        "title": title,
        "input_text": text
        })
    
    return answer['text']

In [14]:
title_counter = 1
entity_bacth = []
relation_bacth = []
relation_bacth_t2c = []
import hashlib

for chapter_title, chapter_chunk in chapter_chunks:
    # title_list = []
    for chunk in chapter_chunk:
        parsed_data = parse_text_structure(chunk)
        # print(parsed_data)
        for i in range(0, len(parsed_data), 2):
            c1 , id, title, level, pid = parsed_data[i]
            c2, content, parent_id = parsed_data[i + 1]

            if len(title) > 20:
                print(title)
            desc = ''
            # if len(content) > 50:
            #     desc = get_decs_from_text(title, content)
            # else:
            #     desc = content
                
            page_content_sha1 = hashlib.sha1(content.encode())
            title_data = {
                "id": id,
                "name": title,
                "level": level,
                "chapter": chapter_title,
                "page_content_sha1": page_content_sha1.hexdigest(),
                "page_content": content,
                "description": desc
            }
            
            entity_bacth.append(title_data)
            
            print(f"ID: {id}, Title: {title}, Level: {level}, Parent ID: {pid}")
            if level != 0:
                relation_data = {
                    "start_id": pid,
                    "end_id": id,
                    "relation": "包含"
                }
                relation_bacth.append(relation_data)
            else:
                relation_data = {
                    "chapter": chapter_title[4:],
                    "end_id": id,
                    "relation": "包含"
                }
                relation_bacth_t2c.append(relation_data)

ID: 1, Title: 制度的概念 【单选】, Level: 0, Parent ID: None
ID: 2, Title: 教育制度的定义与构成要素 【单选、填空】, Level: 0, Parent ID: None
ID: 3, Title: 教育制度的定义, Level: 1, Parent ID: 2
《教育大辞典》的界定是“一个国家各种教育机构的体系，包括学校制度和管理学校的教育行政机构体系”
ID: 4, Title: 《教育大辞典》的界定是“一个国家各种教育机构的体系，包括学校制度和管理学校的教育行政机构体系”, Level: 3, Parent ID: 3
《中国大百科全书》教育卷对教育制度作出了两种阐释
ID: 5, Title: 《中国大百科全书》教育卷对教育制度作出了两种阐释, Level: 3, Parent ID: 3
在大陆和台湾学者联合编写的《教育百科辞典》中，教育制度有两种解释
ID: 6, Title: 在大陆和台湾学者联合编写的《教育百科辞典》中，教育制度有两种解释, Level: 3, Parent ID: 3
ID: 7, Title: 教育制度的构成要素, Level: 1, Parent ID: 2
ID: 8, Title: 规则要素, Level: 2, Parent ID: 7
ID: 9, Title: 物质要素, Level: 2, Parent ID: 7
ID: 10, Title: 组织要素, Level: 2, Parent ID: 7
ID: 11, Title: 学校教育制度 【单选、填空、名词解释】, Level: 0, Parent ID: None
ID: 12, Title: 学校教育制度的含义, Level: 1, Parent ID: 11
ID: 13, Title: 学校教育制度的制约因素, Level: 1, Parent ID: 11
ID: 14, Title: 学校教育制度的变迁, Level: 1, Parent ID: 11
ID: 15, Title: 现代学校教育制度的类型, Level: 1, Parent ID: 11
ID: 16, Title: 双轨型学制, Level: 2, Parent ID: 15
ID: 17, Title: 单轨型学制, Le

In [15]:
relation_bacth_t2c

[{'chapter': '教育制度概述', 'end_id': 1, 'relation': '包含'},
 {'chapter': '教育制度概述', 'end_id': 2, 'relation': '包含'},
 {'chapter': '教育制度概述', 'end_id': 11, 'relation': '包含'},
 {'chapter': '中国教育制度', 'end_id': 19, 'relation': '包含'},
 {'chapter': '中国教育制度', 'end_id': 32, 'relation': '包含'},
 {'chapter': '中国教育制度', 'end_id': 43, 'relation': '包含'},
 {'chapter': '中国教育政策', 'end_id': 49, 'relation': '包含'},
 {'chapter': '中国教育政策', 'end_id': 62, 'relation': '包含'},
 {'chapter': '中国教育政策', 'end_id': 71, 'relation': '包含'},
 {'chapter': '中国教育政策', 'end_id': 76, 'relation': '包含'},
 {'chapter': '学生', 'end_id': 80, 'relation': '包含'},
 {'chapter': '学生', 'end_id': 81, 'relation': '包含'},
 {'chapter': '学生', 'end_id': 84, 'relation': '包含'},
 {'chapter': '学生', 'end_id': 95, 'relation': '包含'},
 {'chapter': '学生发展与教育', 'end_id': 113, 'relation': '包含'},
 {'chapter': '学生发展与教育', 'end_id': 114, 'relation': '包含'},
 {'chapter': '学生发展与教育', 'end_id': 119, 'relation': '包含'},
 {'chapter': '教师职业概述', 'end_id': 143, 'relation': '包含'},
 {'

In [16]:
for entity in entity_bacth:
    match = re.search(r"【(.*?)】", entity["name"])  # 正则匹配 `【xxx】`
    entity["常考题"] = match.group(1) if match else ""  # 提取内容，如果没有就为空
    entity["name"] = re.sub(r"【(.*?)】", "", entity["name"]).strip()  # 去除 `【xxx】` 并去掉多余空格

In [17]:
query_to_create_main_concept = """
UNWIND $batch_data AS data
MERGE (c:`主要概念` {id: data.id})
SET c += data
"""
graph.query(query_to_create_main_concept, params={"batch_data": entity_bacth})

query_to_create_PART_OF_relation = """
    UNWIND $relationshipst2t AS relationship
    MATCH (c:`主要概念` {id: relationship.start_id})
    MATCH (pc:`主要概念` {id: relationship.end_id})
    MERGE (c)-[:包含]->(pc)
    """
graph.query(query_to_create_PART_OF_relation, params={"relationshipst2t": relation_bacth})

query_to_create_PART_OF_relation2 = """
    UNWIND $relationshipst2c AS relationship 
    MATCH (c:`主要概念` {id: relationship.end_id})
    MATCH (pc:`节` {title: relationship.chapter})
    MERGE (pc)-[:包含]->(c)
    """
graph.query(query_to_create_PART_OF_relation2, params={"relationshipst2c": relation_bacth_t2c})

[]